In [80]:
import numpy as np

## Viscosity Functions
From "Fundamentals of Machine Component Design" Section 12, pp 633-635
Reference TBL12-1

Note: 

(F-32)*5/9 = C

6.89 * microreyn = milli-PascalSec  --> 6.89 * microreyn /1000 = PascalSec --> 6.89 * 1000 reyn = PascalSec

In [109]:
# Idek what the fuck that scaling is on the plot. 
# Its like exponential scaling, which I have never even seen before.
# It shows empiral formulas for the basic SAE oils tho
def Visc_SAE_Seireg(Temp, Grade, Metric = False):
    if (Metric):
        Temp = (Temp * 9/5) + 32 # C to F
    match Grade:
        case 10:
            m0 = 0.0158E-6
            b = 1157.5
        case 20:
            m0 = 0.0136E-6
            b = 1271.6
        case 30:
            m0 = 0.0141E-6
            b = 1360.0
        case 40:
            m0 = 0.0121E-6
            b = 1474.4
        case 50:
            m0 = 0.0170E-6
            b = 1509.6
        case 60:
            m0 = 0.0187E-6
            b = 1564.0
    if (not Metric):
        return m0 * np.e**(b / (Temp + 95))    
    else:
        return m0 * np.e**(b / (Temp + 95)) * 6.89 * 1000 # convert back to Metric

# Exponential Fit of the Below Data Source, as found in the GDrive
# https://wiki.anton-paar.com/us-en/engine-oil/
def Visc_SAE_Inter(Temp, Grade, Metric = False):
    match Grade:
        case "30":
            if (Metric): mu = 243.9 * Temp**(-2.24)
            else: mu = 119.4 * Temp**(-3.48)
        case "10W-40":
            if (Metric): 
                if(Temp < 10): # very inaccurate fit below this
                    print("Outside of Interpolation acceptability range")
                    return None
                else: mu = 19.6 * Temp**(-1.56)
            else: 
                if(Temp < 50): # very inaccurate fit below this
                    print("Outside of Interpolation acceptability range")
                    return None
                else: mu = 0.348 * Temp**(-2.25)
    return mu

            

In [194]:
# Sommerfeld Calculation for Naturally Lubricated Journal Bearings
def Sommerfeld(mu, r, c, N, p, echo = False):
    S = (float(r)/float(c))**2 * float(mu) * float(N) / float(p) 
    if(echo): print("Sommerfeld #: " + str(S))
    return S 

# Frictional Coefficent r/c * f calculation
#   Polynomial Fit to readings from Fig 12-18. 
#   4th order chosen for Infinite Case
#   Non-infinite cases not implemented
def FricFactor(S, mu, N, p, r, c, inf = True, echo = True):
    if (inf):
        if (S > 0.5): 
            if (echo): print("Petroff's Equation for Extrapolation")
            f = 2 * np.pi**2 * mu * N / P * r 
        else:
            A = 0.407
            B = 22.7
            C = -86.8
            D = 310
            E = -310
            f = A + B * S + C * S**2 + D * S**3 + E * S**4
    else:
        f = None # not implemented for non-inf
    return f    

# Flow Factor Q/(rcNl) calaculation
#   Fit values to readings from fig 12-19.
#   Non-infinite cases not implemented
def FlowFactor(S, inf = True):
    if(inf):
        if(S < 0.06):
            Q = 37.4 * S - 9.32E-3 # Linear Interpolation 0 -> 0.06, R^2 = 0.997
        elif(S < 0.341):
            Q = 3.59 + 0.456 * np.log(S) # Natural Log Interpolation 0.06 -> 0.4, R^2 = 0.916
        else: # S >= 0.341
            Q = 3.1
    else: Q = None
    return Q 

# Calculation of temperature change via the Temperature Factor (see below)
def TempDelta(Tval, p, Metric = False, iter = 0):
    if (Metric): delT = Tval * p / 0.120E-6 # for Metric
    else: delT = Tval * p / 9.7 # for Imperial

    if iter != 0:
        print("Temperature Delta (" + str(iter) + "): " + str(delT))    
    return delT

# Temperature Factor Evaluation for Infinite L/D ratio only
#   Simplification of formula 12-15
def TempFactorINF(S, mu, N, p, r, c):
    frc = FricFactor(S = S, mu = mu, N = N, p = p, r = r, c = c, inf = True, echo = True)
    Qrcnl = FlowFactor(S = S, inf = True)
    return frc / Qrcnl


# Calculation of Temperature Factor for limited L/D ratio
#   Reference Fig 12-24, EQN 12-16
#   Requires inputs to mu, N, p, c to do interpolation
def TempFactor(S, l, d, mu = None, N = None, p = None, c = None, echo = False):
    r = d / 2
    if(abs(l - d) < .001):
        factT = .349109 + 6.00940 * S + .047467 * S**2
    elif(abs(l/d - .5) < .001):
        factT = .394552 + 6.392527 * S - .036013 * S**2
    elif(abs(l/d - .25) < .001):
        factT = .933828 + 6.437512 * S - .011048 * S**2
    else: 
        if(echo): print("Interpolated Temp Factor") # Reference EQ 12-16

        yinf = TempFactorINF(S = S, mu = mu, N = N, p = p, r = r, c = c)
        c_yinf = -1/8 * (1 - l/d) * (1 - 2 * l/d) * (1 - 4 * l/d)

        y1 = TempFactor(S = S, l = 1, d = 1)
        c_y1 = 1/3 * (1 - 2 * l/d) * (1 - 4 * l/d)

        y12 = TempFactor(S = S, l = 1, d = 2)
        c_y12 = -1 /  4 * (1 - l/d) * (1 - 4 * l/d)
        
        y14 = TempFactor(S = S, l = 1, d = 4)
        c_y14 = 1 / 24 * (1 - l/d) * (1 - 2 * l/d)

        #factT = (l/d)**(-3) * (c_yinf * yinf + c_y1 * y1 + c_y12 * y12 + c_y14 * y14)     
        facT = None
    
    if (echo): print("Temp Factor: " + str(factT))
    return(factT)

# Interchange between Metric and Imperial. "Metric" is if the input is Celsius to begin with, and will convert to F
def ConvertTemp(Temp, Metric = False):
    if (Metric):
        return (Temp * 9/5) + 32
    else:
        return (Temp - 32) * 5/9


In [127]:
# -- Natural Journal Function Definition -- 
# Grade = Oil Grade, as a String ("10W-40") for 
def NaturalJournal(Grade, Tout_guess, T_in, load, rpm, l, r, c, convergence = 0.1, Metric = False):
    p = load / (2 * r * l)
    rps = rpm /60 

    Tavg_guess = (T_in + Tout_guess)/2

    # Determining if it needs to be parsed as interpolated or Seireg for calculating mu

    if(isinstance(Grade, str)): 
        try: 
            mu_guess = Visc_SAE_Inter(Temp = Tavg_guess, Grade = Grade, Metric = Metric)
            method = "Inter"
        except: print("Likely could not Parse Grade")
    if(isinstance(Grade, int)):
        try: 
            mu_guess = Visc_SAE_Seireg(Temp = Tavg_guess, Grade = Grade, Metric = Metric)
            method = "Seireg"
        except: print("Likely could not Parse Grade")
    
    S_guess = Sommerfeld(mu = mu_guess, r = r, c = c, N = rps, p = p)
    delT_guess = TempDelta(Tval = TempFactor(S = S_guess, l = l, d = 2*r, mu = mu_guess, N = rps, p = p, c = c), p = p)

    error = convergence*1.1 # Throwaway to run while loop
    n = 0
    while error > convergence:
        
        # Keeping track of iterative values
        Tavg_prev = Tavg_guess
        mu_prev = mu_guess
        S_prev = S_guess
        delT_prev = delT_guess
        n = n+1
        
        Tout_guess = T_in + delT_guess
        Tavg_guess = T_in + delT_guess/2
        match method:
            case "Inter": mu_guess = Visc_SAE_Inter(Temp = Tavg_guess, Grade = Grade, Metric = Metric)
            case "Seireg": mu_guess = Visc_SAE_Seireg(Temp = Tavg_guess, Grade = Grade, Metric = Metric)
        S_guess = Sommerfeld(mu = mu_guess, r = r, c = c, N = rps, p = p)
        delT_guess = TempDelta(Tval = TempFactor(S = S_guess, l = l, d = 2*r, mu = mu_guess, N = rps, p = p_psi, c = c), p = p)

        error = abs(delT_guess - delT_prev)

    print("Converged on Iteration:" + '\t' + str(n))
    print("Sommerfeld:" + '\t' + str(S_guess))
    print("Outlet Temperature:" + '\t' + str(T_in + delT_guess))
    return(S_guess, T_in + delT_guess)


# EMAE370 Chapter 12: Journal Bearings

## Iterative Approach

In [191]:
Grd = 30 #SAE30

load_lbf = 10E3
rpm = 900
Tin_degF = 180

l_in = 12
r_in = 12

p_psi = load_lbf / (2*r_in * l_in)

c = r_in*2/1000

rps = 900 / 60

Tout_guess = 205
Tavg_guess = (180+205)/2

print("Pressure (psi): " + str(p_psi))

Pressure (psi): 34.72222222222222


In [171]:
Tavg_guess
print(c)

0.024


In [196]:
NaturalJournal(Grade = Grd, Tout_guess = 205, T_in = Tin_degF, load = load_lbf, rpm = rpm, l = l_in, r = r_in, c = c)


Converged on Iteration:	2
Sommerfeld:	0.20274919171300873
Outlet Temperature:	186.0465037493159


(0.20274919171300873, 186.0465037493159)

In [195]:

mu = Visc_SAE_Seireg(Temp = (180+205)/2, Grade = 30, Metric = False)
#print(mu)
#mu = Visc_SAE_Inter(Temp = (180+205)/2, Grade = "30", Metric = False)
S_guess = Sommerfeld(mu = mu, r = r_in, c = c, N = rps, p = p_psi)
print(S_guess)
delT_g = TempDelta(Tval = TempFactor(S = S_guess, l = l_in, d = r_in*2, mu = mu, N = rps, p = p_psi, c =c), p = p_psi)
#print(TempFactor(S = S_guess, l = l_in, d = r_in*2, mu = mu, N = rps, p = p_psi, c =c))
#print(delT_g)

1.4968389405746452
5.358100445928713


In [43]:
mu = 1.5981621311834866e-06
r = 4
c = 0.008
N = 15
p = 312.5
(r/c)**2 * mu * N / p
#Sommerfeld(mu= 1.5981621311834866e-06, r = 4, c = .008, N = 15, p = 312.5)

0.01917794557420184

In [ ]:
# Guess Temperature
Tout_g = 205
Tavg_g = (Tin_degF + Tout_g)/2
print("Guess Film Average Temp, degF: " + str(Tavg_g))

In [ ]:
Tout = Tout_g

error = 1.1 # throwaway value
firstIterFlag = True
convergence_res = .1 # deg F
n = 1
while error > convergence_res:
    if firstIterFlag:   
        Tavg = (Tin_degF + Tout)/2
        mu = Visc_SAE_Seireg(Temp = Tavg, Grade = Grd, Metric = False)
        print("Viscosity: " + str(mu))
        S = Sommerfeld(mu = mu, r = r_in, c = c, N = rps, p = p_psi)
        delT = TempDelta(Tval = TempFactor(S = S, l = l_in, d = 2*r_in), p = p_psi, iter = n)
        firstIterFlag = False
    
    n = n+1
    # Storing for comparison to exit while loop
    Tavg_prev = Tavg
    mu_prev = mu 
    S_prev = S 
    delT_prev = delT 

    Tout = Tin_degF + delT 
    Tavg = Tin_degF + delT/2
    mu = Visc_SAE_Seireg(Temp = Tavg, Grade = Grd, Metric = False)
    S = Sommerfeld(mu = mu, r = r_in, c = c, N = rps, p = p_psi)
    delT = TempDelta(Tval = TempFactor(S = S, l = l_in, d = 2*r_in), p = p_psi, iter = n)

    error = abs(delT - delT_prev) 
    if (error <= convergence_res):
        print("S ~ last iteration Sommerfeld: " + str(S) + '\n' + "Tout ~ last iteration Temp: " + str(Tin_degF + delT))
        output = (S, Tin_degF + delT)



## Manual Approach

In [174]:
## SAE 30

load_lbf = 10E3
rpm = 900
Tin_degF = 180

l_in = 12
r_in = 12

# --- #

p_psi = load_lbf / (2*r_in * l_in)

c = r_in*2/1000
rps = 900 / 60

print("Pressure (psi): " + str(p_psi))

Pressure (psi): 34.72222222222222


In [175]:
Tout_1 = 205
Tavg_1 = (Tin_degF + Tout_1)/2
print("Guess Film Average Temp, degF: " + str(Tavg_1))

Guess Film Average Temp, degF: 192.5


In [176]:
mu_1 = 1.4E-6

In [185]:
S_1 = Sommerfeld(mu = mu_1, r = r_in, c = c, N = rps, p = p_psi)
print(S_1)

0.1512


In [188]:
delT_1 = TempDelta(Tval = TempFactor(S = S_1, l = l_in, d = 2*r_in), p = p_psi, iter = 1)

T_2 = Tin_degF + delT_1
print(T_2)

Temperature Delta (1): 393599182.1068519
393599362.1068519


In [ ]:
Tavg_2 =  Tin_degF + delT_1/2
print("Guess Film Average Temp, degF: " + str(Tavg_2))

In [ ]:
mu_2 = 1.8E-6
S_2 = Sommerfeld(mu = mu_2, r = r_in, c = c, N = rps, p = p_psi)
delT_2 = TempDelta(Tval = TempFactor(S = S_2, l = l_in, d = 2*r_in), p = p_psi, iter = 2)
T_3 = Tin_degF + delT_2
print(T_3)

In [ ]:
Tavg_3 =  Tin_degF + delT_2/2
print("Guess Film Average Temp, degF: " + str(Tavg_3))

In [ ]:
mu_3 = 1.8E-6
S_3 = Sommerfeld(mu = mu_3, r = r_in, c = c, N = rps, p = p_psi)
delT_3 = TempDelta(Tval = TempFactor(S = S_3, l = l_in, d = 2*r_in), p = p_psi, iter = 3)
T_4 = Tin_degF + delT_3

In [ ]:
Tout = Tin_degF + delT_3
print(Tout)

In [ ]:
h_0 = c * .8
print("h_0: " + str(h_0))
f = c/r_in * 1.4
print("f: " + str(f))
Q = 5.85 * r_in * c * rps * l_in
print("Qfull: " + str(Q))
Qs = .94 * Q 
print("Side Flow Qs: " + str(Qs))
